To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Llama-3 8b is trained on a crazy 15 trillion tokens! Llama-2 was 2 trillion.**

Use our [Llama-3 8b Instruct](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing) notebook for conversational style finetunes.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path

workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("workding dir:", workding_dir)

workding dir: /home/inflaton/code/projects/courses/novel-translation


In [3]:
from dotenv import find_dotenv, load_dotenv

found_dotenv = find_dotenv(".env")

if len(found_dotenv) == 0:
    found_dotenv = find_dotenv(".env.example")
print(f"loading env vars from: {found_dotenv}")
load_dotenv(found_dotenv, override=True)

loading env vars from: /home/inflaton/code/projects/courses/novel-translation/.env


True

In [4]:
import os

model_name = os.getenv("MODEL_NAME")
token = os.getenv("HF_TOKEN") or None
load_in_4bit = os.getenv("LOAD_IN_4BIT") == "true"
local_model = os.getenv("LOCAL_MODEL")
hub_model = os.getenv("HUB_MODEL")
num_train_epochs = int(os.getenv("NUM_TRAIN_EPOCHS") or 0)
data_path = os.getenv("DATA_PATH")

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)

model_name, load_in_4bit, local_model, hub_model, max_seq_length, num_train_epochs, dtype, data_path

('unsloth/Qwen2-7B-Instruct',
 True,
 'models/Qwen2-7B-Instruct-MAC-',
 'Qwen2-7B-Instruct-MAC-',
 2048,
 10,
 None,
 'datasets/mac/mac.tsv')

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

In [5]:
%%time

from translation_engine import *

model, tokenizer = load_model(model_name, load_in_4bit)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


[nltk_data] Downloading package wordnet to /home/inflaton/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/inflaton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/inflaton/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


==((====))==  Unsloth: Fast Qwen2 patching release 2024.5
   \\   /|    GPU: NVIDIA GeForce RTX 4080 Laptop GPU. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 26 s, sys: 27.4 s, total: 53.4 s
Wall time: 8min 8s


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [6]:
%%time

model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth 2024.5 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


CPU times: user 15.5 s, sys: 0 ns, total: 15.5 s
Wall time: 2.13 s


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [7]:
import os
from translation_engine import *

datasets = load_translation_dataset(data_path, tokenizer)

loading train/test data files
DatasetDict({
    train: Dataset({
        features: ['chinese', 'english', 'text', 'prompt'],
        num_rows: 4528
    })
    test: Dataset({
        features: ['chinese', 'english', 'text', 'prompt'],
        num_rows: 1133
    })
})


In [8]:
datasets["train"][0], datasets["test"][0]

({'chinese': '全仗着狐仙搭救。',
  'english': 'Because I was protected by a fox fairy.',
  'text': '<|im_start|>system\nYou are an expert in translating Chinese into English.<|im_end|>\n<|im_start|>user\nTranslate from Chinese to English.\n全仗着狐仙搭救。<|im_end|>\n<|im_start|>assistant\nBecause I was protected by a fox fairy.<|im_end|>',
  'prompt': '<|im_start|>system\nYou are an expert in translating Chinese into English.<|im_end|>\n<|im_start|>user\nTranslate from Chinese to English.\n全仗着狐仙搭救。<|im_end|>\n<|im_start|>assistant\n'},
 {'chinese': '老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。',
  'english': 'Old Geng picked up his shotgun, squinted, and pulled the trigger. Two sparrows crashed to the ground like hailstones as shotgun pellets tore noisily through the branches.',
  'text': '<|im_start|>system\nYou are an expert in translating Chinese into English.<|im_end|>\n<|im_start|>user\nTranslate from Chinese to English.\n老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。<|im_end|>

In [9]:
datasets["train"][1000], datasets["test"][1000]

({'chinese': '周瑞家的道：“太太说：‘他们原不是一家子； 当年他们的祖和太老爷在一处做官，因连了宗的。',
  'english': "'She said they don't really belong to the family but were adopted into the clan years ago when your grandfather and theirs were working in the same office.",
  'text': "<|im_start|>system\nYou are an expert in translating Chinese into English.<|im_end|>\n<|im_start|>user\nTranslate from Chinese to English.\n周瑞家的道：“太太说：‘他们原不是一家子； 当年他们的祖和太老爷在一处做官，因连了宗的。<|im_end|>\n<|im_start|>assistant\n'She said they don't really belong to the family but were adopted into the clan years ago when your grandfather and theirs were working in the same office.<|im_end|>",
  'prompt': '<|im_start|>system\nYou are an expert in translating Chinese into English.<|im_end|>\n<|im_start|>user\nTranslate from Chinese to English.\n周瑞家的道：“太太说：‘他们原不是一家子； 当年他们的祖和太老爷在一处做官，因连了宗的。<|im_end|>\n<|im_start|>assistant\n'},
 {'chinese': '“听到了吗？',
  'english': "'Did you hear that?'",
  'text': "<|im_start|>system\nYou are an expert in translating Chinese i

In [10]:
%%time

prompt1 = datasets["test"]["prompt"][1000]
print(prompt1)
print("--" * 20)
test_model(model, tokenizer, prompt1)

<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
“听到了吗？<|im_end|>
<|im_start|>assistant

----------------------------------------
<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
“听到了吗？<|im_end|>
<|im_start|>assistant


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [11]:
%%time

predictions = eval_model(model, tokenizer, datasets["test"])

  0%|          | 0/1133 [00:00<?, ?it/s]


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [12]:
calc_metrics(datasets["test"]["english"], predictions, debug=True)

NameError: name 'predictions' is not defined

In [ ]:
save_results(
    model_name,
    "results/mac-results.csv",
    datasets["test"],
    predictions,
    debug=True,
)

   Unnamed: 0                                            chinese  \
0           0  老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞...   

                                             english  \
0  Old Geng picked up his shotgun, squinted, and ...   

              unsloth/Qwen2-0.5B-Instruct(finetuned)  \
0  Old Geng lifted his rifle and narrowed his eye...   

                         unsloth/Qwen2-1.5B-Instruct  
0  Old Geng took up his gun, squinted one of its ...  


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=datasets["train"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=num_train_epochs,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=100,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/4528 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
import torch

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4080 Laptop GPU. Max memory = 11.994 GB.
3.633 GB of memory reserved.


In [ ]:
%%time

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,528 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 5,660
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
100,1.919100
200,1.774900
300,1.722600
400,1.721900
500,1.695700
600,1.612500
700,1.473700
800,1.518000
900,1.452100
1000,1.454900


CPU times: user 1h 23min 59s, sys: 8min 44s, total: 1h 32min 43s
Wall time: 1h 32min 45s


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

5564.5261 seconds used for training.
92.74 minutes used for training.
Peak reserved memory = 4.152 GB.
Peak reserved memory for training = 0.519 GB.
Peak reserved memory % of max memory = 34.617 %.
Peak reserved memory for training % of max memory = 4.327 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
%%time

prompt1 = datasets["test"]["prompt"][0]
print(prompt1)
print("--" * 20)
test_model(model, tokenizer, prompt1)

<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。<|im_end|>
<|im_start|>assistant

----------------------------------------
<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。<|im_end|>
<|im_start|>assistant
Old Geng raised the pistol, squinted one eye, squeezed the trigger, and let a shower of jumbo pigeons drop down from skyward, coursing through the willows as though carried on silkworm tails, tossing tin cans in the air as they fell.<|im_end|>
CPU times: user 3.71 s, sys: 352 ms, total: 4.07 s
Wall time: 4.04 s


In [ ]:
print(datasets["test"]["english"][0])

Old Geng picked up his shotgun, squinted, and pulled the trigger. Two sparrows crashed to the ground like hailstones as shotgun pellets tore noisily through the branches.


In [ ]:
%%time

predictions = eval_model(model, tokenizer, datasets["test"])

100%|██████████| 1133/1133 [34:09<00:00,  1.81s/it] 

CPU times: user 30min 48s, sys: 3min 21s, total: 34min 10s
Wall time: 34min 9s


In [ ]:
calc_metrics(datasets["test"]["english"], predictions, debug=True)

[nltk_data] Downloading package wordnet to /home/inflaton/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/inflaton/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/inflaton/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'accuracy': 0.00264783759929391,
 'correct_ids': [147, 170, 194],
 'meteor': {'meteor': 0.35503843183028994},
 'bleu_scores': {'bleu': 0.09734851870184895,
  'precisions': [0.38486636126948554,
   0.12903115371448134,
   0.05879839025606325,
   0.030757244091566802],
  'brevity_penalty': 1.0,
  'length_ratio': 1.0050679032792316,
  'translation_length': 30343,
  'reference_length': 30190},
 'rouge_scores': {'rouge1': 0.3809259470501297,
  'rouge2': 0.1543849804952549,
  'rougeL': 0.32312000381943484,
  'rougeLsum': 0.32320284655253784}}

In [ ]:
from translation_engine import save_results

save_results(model_name + "(finetuned)", "results/mac-results.csv", datasets["test"], predictions, debug=True)

   index                                            chinese  \
0      0  老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞...   

                                             english  \
0  Old Geng picked up his shotgun, squinted, and ...   

              unsloth/Qwen2-0.5B-Instruct(finetuned)  \
0  Old Geng lifted his rifle and narrowed his eye...   

                         unsloth/Qwen2-1.5B-Instruct  \
0  Old Geng took up his gun, squinted one of its ...   

              unsloth/Qwen2-1.5B-Instruct(finetuned)  
0  Old Geng raised the rifle, squeezed one tiny t...  


<a name="Save"></a>
### Saving, uploading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

In [ ]:
def save_model(model, tokenizer, save_method, publish=True):
    model.save_pretrained_merged(
        local_model + save_method,
        tokenizer,
        save_method=save_method,
    )

    if publish:
        model.push_to_hub_merged(
            hub_model + save_method,
            tokenizer,
            save_method=save_method,
            token=token,
        )

In [ ]:
save_model(model, tokenizer, "merged_4bit_forced")

Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...
Done.
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 10 minutes for Llama-7b... Done.
Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...
Done.
Unsloth: Saving 4bit Bitsandbytes model. Please wait...


README.md:   0%|          | 0.00/575 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/727M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/581 [00:00<?, ?B/s]

Saved merged_4bit model to https://huggingface.co/Qwen2-1.5B-Instruct-MAC-merged_4bit_forced


In [ ]:
%%time

# Empty VRAM
del model
del trainer

# clear memory
import torch
torch.cuda.empty_cache()

# garbage collect
import gc
gc.collect()
gc.collect()

CPU times: user 365 ms, sys: 1.58 ms, total: 367 ms
Wall time: 1.08 s


0

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>